In [1]:
import libvirt
import sys
import threading
import socket
import ast
from datetime import datetime
import time
import hashlib
from random import shuffle
import struct

In [2]:
conn = libvirt.open('qemu:///system')
conn.getHostname()

'ankita-ThinkPad-E490'

In [3]:
conn.getMaxVcpus('qemu')

16

In [4]:
#print(conn.getSysinfo()) //get system information
conn.listDomainsID()

[3]

In [5]:
'''
See information about active VMs
'''
def active_VM_info():
    domainIDs = conn.listDomainsID()
    for domainID in domainIDs:
        domain = conn.lookupByID(domainID)
        uuid = domain.UUIDString()
        print("INFORMATION FOR DOMAIN :",uuid)
        print(domain.info())
        print(domain.OSType())
        print(domain.getCPUStats(total=5))
        print(domain.jobStats())
    

In [6]:
active_VM_info()

INFORMATION FOR DOMAIN : eaa5870f-be68-4e93-9274-d1351a3fef0a
[1, 1048576, 1048576, 1, 10930000000]
hvm
[{'cpu_time': 10948265289, 'user_time': 910000000, 'system_time': 1640000000}]
{'type': 0}


In [7]:
'''
Spawn a new VM
'''
def spawn():
    dom0 = conn.lookupByName('ubuntu18.04-3')
    dom0.create()

In [8]:
#spawn()

In [9]:
'''
Calculate CPU usage
'''
def calc_CPU_usage(VM_name):
    dom0 = conn.lookupByName(VM_name)
    cpu_time_in = dom0.getCPUStats(1)[0]['cpu_time']
    time.sleep(1)
    avg_cpututil =0;
    for i in range(0,5):
        cpu_time_fin = dom0.getCPUStats(1)[0]['cpu_time']
        #print(dom0.getCPUStats(1))
        delta = cpu_time_fin-cpu_time_in
        avg_cpututil +=delta
        cpu_time_in = cpu_time_fin
        time.sleep(1)
    return 100*avg_cpututil/(5*1000000000)

In [10]:
#print(calc_CPU_usage('ubuntu18.04'))
#dom0.getCPUStats(1)[0]['cpu_time']
print(calc_CPU_usage('ubuntu18.04-2'))

0.34888868


Normally for a process on Linux, the overall CPU usage is simply the sum of the time spent in userspace and the time spent on kernel space. So naively one would have expected user_time + system_time to equal cpu_time. What I've discovered is that Linux tracks time spent by vCPU threads executing guest code separately from either userspace or kernelspace time.

Thus cpu_time == user_time + system_time + guest_time

So you can think of system_time + user_time as giving the overhead of QEMU / KVM on the host side. And cpu_time - (user_time + system_time) as giving the actual amount of time the guest OS was running its CPUs.

To calculate CPU usage, you probably just want to record cpu_time every N seconds and calculate the delta between two samples. eg usage % = 100 * (cpu_time 2 - cpu_time 1) / N

In [11]:

# f = open("abc.txt", "r")
# xmlconfig = f.read()
# dom = conn.createXML(xmlconfig, 0)
# if dom == None:
#     print('Failed to create a domain from an XML definition.', file=sys.stderr)
#     exit(1)

# '''
# The xml file used above does not contain any disk storage
# Will need to declare it later
# '''
# # if dom.create() < 0:
# #     print('Can not boot guest domain.', file=sys.stderr)
# #     exit(1)
# print('Guest '+dom.name()+' has booted', file=sys.stderr)


In [12]:

'''
To notify client about new VM
'''

IPAddr = "127.0.1.1"
port = 10010

client_ip ="0.0.0.0"
#client_ip ="192.168.122.186"
client_port=10011

def notify_client():
    
    while True:
        k=0
        connected_seeds=0
        node = ""
        for i in range(0,5):
            try: #get other peers info from seed node(s)
                s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                s.connect((client_ip, client_port))
                print("Connected to client node:", client_ip)
                values = (b'news', port)
                packer = struct.Struct('4s I')
                packed_data = packer.pack(*values)
                for j in range(0,5):
                    s.sendall(packed_data)
                    time.sleep(0.12)
#                 data = s.recv(1024)
#                 data = data.decode('utf-8')
                s.close()
                break
            except:
                print("Error while connection to seed node or fetching Peer list:", sys.exc_info()[1])
                k+=1
                s.close()
                continue
        

        break
        print("Retrying after 5 sec...")
        time.sleep(5)

In [ ]:
while True:
    Cpu_usage = calc_CPU_usage('ubuntu18.04-2')
    print(Cpu_usage)
    if(Cpu_usage>50):
        print("Time to spawn a new VM!")
        spawn()
        time.sleep(70) #For VM to boot up
        print("Time to notify client!")
        notify_client()
        #break

0.62762782
16.34337438
0.8771427
3.5413681
2.52537826
0.52512212
0.41729368
0.32781594
0.38093788
0.30925676
0.37389318
11.62850062
40.40050074
40.37079094
36.54009494
36.20454532
39.25146908
40.53321284
36.23663618
0.35983152
23.78799512
72.67900368
Time to spawn a new VM!
Time to notify client!
Connected to client node: 0.0.0.0
36.07900092
36.78379616
35.04654732
35.8309373
33.9348292
39.8219842
34.61792714
36.33364084
34.14478488
35.82139308
33.97719902
37.14034266
35.38417876
35.91512582
33.60662162
40.4562288
36.29727552
37.8225885
37.0442403
36.77439322
37.67211824
33.2118388
37.95767188
32.7045908
37.90370874
38.59375804
32.83917798
38.45666638
33.40127318
38.1755402
38.64435982
33.10122944
38.51670688
33.30982968
37.14951554
40.15388934
35.7316332
36.77028278
36.8419362
36.83284466
35.95420314
36.12800204
37.2329999
34.45405794
35.78842962
37.16992128
36.51338588
37.06020332
36.02038232
38.07925134
33.66226338
36.6036449
36.53597598
36.34123488
36.1671322
40.0601737
33.23126776